In [13]:
from datetime import datetime, timedelta
from pandas import DataFrame
from models.Payload import REGION, RegPayload, AffectedPayload
from models.Source import DataSource as SecApi
from utils.transforms import str2date
from urllib.parse import urlencode as url
#cspell:disable

#input
region:REGION='Valparaiso'
#total regional clients
users = SecApi()
q = users.request('reg_clients',RegPayload(region=region))
clients = int(q[0]['CLIENTES'])
print(f"{"usuarios regionales":<22}",':',f"{clients:,.0f} clientes")

#data
curr = SecApi()
curr.request('affected_detail',AffectedPayload()) 
curr_df=curr.dataframe(FECHA_INT_STR=str2date)
curr_df= curr_df[curr_df['NOMBRE_REGION']==region].drop(columns=['HORA','DIA','MES','ANHO'])
affected = curr_df['CLIENTES_AFECTADOS'].sum()

print(f'{'usuarios sin servicio':<22}',':',f'{affected:,.0f}',' clientes',f'({affected/clients:.2%})')
print(f'{'fecha reporte':<22}',':',curr.report_date.strftime("%d/%m/%Y %H:%M"))

res:DataFrame = curr_df.groupby(['NOMBRE_COMUNA','NOMBRE_EMPRESA','FECHA_INT_STR'],as_index=False)['CLIENTES_AFECTADOS'].sum().sort_values(['CLIENTES_AFECTADOS'],ascending=False)

#cspell:disable
res:DataFrame = res.rename(columns={
    'FECHA_INT_STR':'INCIDENTE',
    'CLIENTES_AFECTADOS':'SIN_SERVICIO',
    'NOMBRE_EMPRESA':'DISTRIBUIDORA',
    'NOMBRE_COMUNA':'COMUNA'
    })

#days w/o service
res['TIEMPO_SIN_SERVICIO'] = res['INCIDENTE'].apply(lambda event_date: datetime.now()-event_date+timedelta(days=0))
res

usuarios regionales       : 780,563 clientes
usuarios sin servicio     : 1,017  clientes (0.13%)
fecha reporte             : 22/08/2024 09:37


,COMUNA,DISTRIBUIDORA,INCIDENTE,SIN_SERVICIO,TIEMPO_SIN_SERVICIO
10,Viña del Mar,CHILQUINTA,2024-02-19,513,185 days 11:32:43.289071
1,Calle Larga,CHILQUINTA,2024-08-22,281,0 days 11:32:43.289071
7,Quilpue,CHILQUINTA,2024-08-22,107,0 days 11:32:43.289071
5,Quilpue,CHILQUINTA,2024-02-19,83,185 days 11:32:43.289071
3,Los Andes,CHILQUINTA,2024-08-22,20,0 days 11:32:43.289071
9,Viña del Mar,CGE,2024-08-22,7,0 days 11:32:43.289071
8,San Antonio,CGE,2024-08-22,2,0 days 11:32:43.289071
0,Cabildo,CGE,2024-08-22,1,0 days 11:32:43.289071
2,La Ligua,CGE,2024-08-22,1,0 days 11:32:43.289071
4,Papudo,CGE,2024-08-22,1,0 days 11:32:43.289071


In [3]:
res_by_city:DataFrame = curr_df.groupby(['NOMBRE_COMUNA'],as_index=False)['CLIENTES_AFECTADOS']\
    .sum().sort_values(['CLIENTES_AFECTADOS'],ascending=False)
res_by_city

,NOMBRE_COMUNA,CLIENTES_AFECTADOS
4,Viña del Mar,515
0,Calle Larga,281
2,Quilpue,191
1,Los Andes,20
3,San Antonio,2


In [ ]:
from datetime import datetime
delta = datetime.now()-datetime.strptime("19-06-2024","%d-%m-%Y")
str(delta)